# Project 4

This project applies programming skills acquired during the course to re-analyze microscopy data using Cellpose, comparing its segmentation results with those from the Nessys algorithm.

The authors of the PLOS Biology paper, "Nessys: A new set of tools for the automated detection of nuclei within intact tissues and dense 3D cultures" (Blin et al., 2019) introduced a novel nuclear segmentation method. Their images were processed using Nessys, an extensible Java-based automated cell nuclei identification software. The images and analytical results are available in the Image Data Resource (IDR):
- <https://doi.org/10.1371/journal.pbio.3000388>
- <https://idr.openmicroscopy.org/webclient/?show=project-801>

The IDR team has also converted the dataset into OME-Zarr format. Unlike Nessys, Cellpose was not used in the original study. 

## Local TIFF files

The dataset is available in the IDR, but first, we are going to download the B4_C3.tif file from the IDR and save it locally. We will then use Cellpose to segment the nuclei in the image and compare the results with those obtained using Nessys.

### 1. Install dependencies from [PyPI](https://pypi.org/)

In [1]:
# Install dependencies
!pip install -r requirements.txt

In [2]:
# Import libraries
import bioio
import cellpose
import cellpose.models
import dask
from ipywidgets import *
import matplotlib.pyplot as plt
import numpy

### 2. Download C3_B4.tif from IDR

Using the filepath, it is possible to download locally the image by running from a terminal:

```bash
wget https://ftp.ebi.ac.uk/pub/databases/IDR/idr0062-blin-nuclearsegmentation/20190429-ftp/Blastocysts/B4_C3.tif
```

or, alternatively, by using the Python `ftputil` library:

In [3]:
import ftputil
import os

files = [
    "pub/databases/IDR/idr0062-blin-nuclearsegmentation/20190429-ftp/Blastocysts/B4_C3.tif",
    "pub/databases/IDR/idr0062-blin-nuclearsegmentation/20190429-ftp/Blastocysts/B4_C3_Manual.tif",
]

with ftputil.FTPHost("ftp.ebi.ac.uk", "anonymous") as host:
    names = host.listdir(host.curdir)
    for name in names:
        if name == "pub":  # only check the IDR data
            for file in files:
                filepath = os.path.join("data", os.path.basename(file))
                if host.path.isfile(file):
                    host.download(file, filepath)

### 3. Try to open in BioIO

The `bioio` package requires `bioio-tifffile` plugin to be able to load the image as downloaded.

In [4]:
filepath = os.path.join("data", "B4_C3.tif")
labelspath = os.path.join("data", "B4_C3_Manual.tif")

In [5]:
# Load the image
images = bioio.BioImage(filepath)

# Load the label
labels = bioio.BioImage(labelspath)

# (T, C, Z, Y, X)
print(images.dims)
print(images.shape)

<Dimensions [T: 1, C: 2, Z: 257, Y: 210, X: 253]>
(1, 2, 257, 210, 253)


In [6]:
def update(z=0, c=0):
    fig = plt.figure(figsize=(10, 10))
    plt.imshow(images.data[0, c, z, :, :])
    plt.tight_layout()
    fig.canvas.flush_events()


interact(
    update,
    z=widgets.IntSlider(
        value=0,
        min=0,
        max=images.data.shape[2] - 1,
        step=1,
        description="Select T",
        continuous_update=False,
    ),
    c=widgets.IntSlider(
        value=0,
        min=0,
        max=images.data.shape[1] - 1,
        step=1,
        description="Select C",
        continuous_update=False,
    ),
)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Select T', max=256), IntSlider(…

<function __main__.update(z=0, c=0)>

### 4. Select two Z-planes around the middle of the stack

In [7]:
# Define the Z-planes to select
Z = [89, 90, 91]

# Select two Z-planes (indices 89 and 90) and stack them along a new axis
selected_images = numpy.stack([images.data[0, :, z, :, :] for z in Z], axis=0)

print(f"BioIO shape: {images.data.shape}")
print(f"Selected images shape: {selected_images.shape}")


BioIO shape: (1, 2, 257, 210, 253)
Selected images shape: (3, 2, 210, 253)


### 5. Process the selected planes using Cellpose API with the default cyto model

In [8]:
# Convert the NumPy image data to a Dask array
dask_img = dask.array.from_array(
    selected_images,
    chunks=(
        1,
        selected_images.shape[1],
        selected_images.shape[2],
        selected_images.shape[3],
    ),
)

# Load the Cellpose model: cyto
model = cellpose.models.Cellpose(model_type="cyto")

# Delayed function for segmentation
@dask.delayed
def segment(image):
    masks, _, _, _ = model.eval(image, diameter=30, channels=[0, 1])
    return masks

In [9]:
%%time
# Perform segmentation with Dask
results = dask.compute(*[segment(plane) for plane in dask_img])

CPU times: user 21.5 s, sys: 4.59 s, total: 26 s
Wall time: 3.44 s


### 6. Display the segmentation results

We will use matplotlib to display the segmentation results. The output will show the Cellpose segmentation alongside the Nessys ground truth labels for comparison.

In [10]:
def update(z=0):
    fig = plt.figure(figsize=(10, 10))
    fig, ax = plt.subplots(1, 3, figsize=(10, 5))

    # Original Image
    ax[0].imshow(dask_img[0][z], cmap="Blues")
    ax[0].imshow(dask_img[1][z], cmap="Reds")
    ax[0].set_title(f"Original - {Z[z] + 1}")
    ax[0].axis("off")
    
    # Segmented Image (Overlay)
    ax[1].imshow(dask_img[0][z], cmap="gray")
    ax[1].imshow(results[z], alpha=0.5, cmap="jet")
    ax[1].set_title(f"Segmentation - {Z[z] + 1}")
    ax[1].axis("off")
    
    # Labels Image
    ax[2].imshow(labels.data[0, 0, Z[z], :, :], cmap="jet")
    ax[2].set_title(f"Nessys' labels - {Z[z] + 1}")
    ax[2].axis("off")

    plt.tight_layout()
    fig.canvas.flush_events()

interact(
    update,
    z=widgets.IntSlider(
        value=0,
        min=0,
        max=len(results) - 1,
        step=1,
        description="Select T",
        continuous_update=False,
    ),
)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Select T', max=2), Output()), _…

<function __main__.update(z=0)>

## OME-Zarr files

In this step, we will use the OME-Zarr format to analyze [6001247.zarr](https://uk1s3.embassy.ebi.ac.uk/idr/zarr/v0.1/6001247.zarr/0/) file. We will load the image using Dask and Cellpose and compare the results with the original Nessys segmentation.